In [1]:
import os
import difflib
from tqdm import tqdm
import pandas as pd
import re
import sys
import json
import base64
import hashlib
import requests
from typing import Union
from requests.auth import HTTPBasicAuth
from requests.exceptions import HTTPError

In [2]:
project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

['C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner\\src',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner']

In [3]:
from config.config import config
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
config['user_1C'] = os.getenv('user_1C')
config['password_1C'] = os.getenv('password_1C')

In [5]:
auth = HTTPBasicAuth(config['user_1C'], config['password_1C'])

### Получение списка номеров счетов

In [6]:
# Получение списка номеров счетов за период времени

# месяцы 04-06 уже есть (excel от экономистов)
# месяцы 07-08 - дообучено
month1 = '11'
month2 = '12'

date_1 = f'01-{month1}-2024'
date_2 = f'01-{month2}-2024'

deal_numbers_list = requests.get(fr'http://10.10.0.10:81/ca/hs/interaction/InvoicesByDate/{date_1}/{date_2}', auth=auth).json()
print(f"length: {len(deal_numbers_list)}")

print(f"deal_numbers: {deal_numbers_list[0:3]}")

length: 2102
deal_numbers: ['0С000000000000142499', '0С000000000000142511', '0С000000000000142641']


### Получение деталей по данным счетам

```
[
{"СтрокиСчета": ["Услуга": , "УслугаКод": , "ТранспортнаяПозиция": , "Количество": , "Цена: "], "ФайлыСчета": [..]},
{"СтрокиСчета": [..], "ФайлыСчета": [..]},
..
]
```

In [ ]:
# Получение деталей счетов
# ЗДЕСЬ ДОБАВИТЬ НОМЕР СЧЕТА К КАЖДОМУ СЛОВАРЮ ДЕТАЛЕЙ

numbers = []
for number in tqdm(deal_numbers_list[100:]):
    try:
        res = requests.get(fr'http://10.10.0.10:81/ca/hs/interaction/InvoiceDetailsByNumber/{number}', auth=auth).json()
    except:
        print(number)
        continue
    numbers.append(res)

 17%|████████████▉                                                                | 336/2000 [20:29<1:19:04,  2.85s/it]

0С000000000000145487


 17%|█████████████                                                                | 338/2000 [20:35<1:21:43,  2.95s/it]

0С000000000000145489


 38%|█████████████████████████████▎                                               | 760/2000 [44:07<1:03:33,  3.08s/it]

...

 46%|███████████████████████████████████▎                                         | 917/2000 [53:04<1:07:58,  3.77s/it]

0С000000000000146038


 49%|█████████████████████████████████████▊                                       | 983/2000 [56:47<1:10:31,  4.16s/it]

0С000000000000146155


 76%|█████████████████████████████████████████████████████████▍                  | 1510/2000 [1:26:53<30:36,  3.75s/it]

In [8]:
numbers[0]

{'СтрокиСчета': [{'Услуга': 'Предоплата ',
   'УслугаКод': 'ТК-004931',
   'ТранспортнаяПозиция': 'ARKU8444548',
   'Количество': 1,
   'Цена': 72466.21}],
 'ФайлыСчета': []}

In [9]:
# Запись деталей в файл

file = os.path.join(config['BASE_DIR'], 'config', 'generated_services', f'result_numbers_{month1}.json')
with open(file, 'w', encoding='utf-8') as f:
    json.dump(numbers, f, ensure_ascii=False, indent=4)

### Фильтрация деталей

In [6]:
# Получение списка частых услуг

file = os.path.join(config['BASE_DIR'], 'config', 'freq_services.json')
with open(file, 'r', encoding='utf-8') as f:
    freq_services = json.load(f)


def remove_special_characters(text):
    response = re.sub(r'[^\w\s]', ' ', text.lower())
    return re.sub(r'(\s{2,}|\n)', ' ', response)
    
freq_services = [remove_special_characters(service) for service in freq_services]

In [7]:
# Определение функций очистки

# hash

def calculate_hash(file_path):
    # Инициализация хэш-объекта MD5
    md5_hash = hashlib.md5()

    # Открываем файл в бинарном режиме для чтения
    with open(file_path, "rb") as f:
        # Чтение файла блоками по 4096 байт (можно настроить)
        for byte_block in iter(lambda: f.read(4096), b""):
            md5_hash.update(byte_block)

    # Возвращаем хэш-сумму в виде шестнадцатеричной строки
    return md5_hash.hexdigest()


def is_single_hash(lst: list[str]) -> str | None:
    """
    lst: result['ФайлыСчета'] from result = requests.get(fr'http://.../InvoiceDetailsByNumber/{deal}', auth=auth).json()
    """
    if not list:
        return None
        
    res = [calculate_hash(file) for file in lst]
        
    if len(set(res)) == 1:
        return lst[0]


# frequent services

def is_all_services_freq(lst: list, freq_services: list) -> bool:
    """
    lst: result['СтрокиСчета'] from result = requests.get(fr'http://.../InvoiceDetailsByNumber/{deal}', auth=auth).json()
    freq_services: list of frequently used services (cleared)
    """
    services = [d['Услуга'] for d in lst]
    clear_services = [remove_special_characters(service) for service in services]
    
    if set(clear_services).issubset(freq_services):
        return True
    else:
        return False    

In [10]:
# неотфильтрованные детали

file = os.path.join(config['BASE_DIR'], 'config', 'generated_services', f'result_numbers_{month1}.json')
with open(file, 'r', encoding='utf-8') as f:
    result = json.load(f)

print(len(result))

1995


In [11]:
# отфильтрованные детали

filtered_response_list = []
for res in tqdm(result):
    
    raws = res['СтрокиСчета']
    files = res['ФайлыСчета']
    
    if is_all_services_freq(raws, freq_services):
        # print('freq')
        continue
    if not is_single_hash(files):
        # print('hash')
        continue

    filtered_response_list.append(res)

print(f"{len(deal_numbers_list)} --> {len(filtered_response_list)}")

100%|██████████████████████████████████████████████████████████████████████████████| 1995/1995 [03:47<00:00,  8.75it/s]

2100 --> 384


In [ ]:
filtered_response_list[0]

In [13]:
# Запись отфильтрованных деталей в файл

file = os.path.join(config['BASE_DIR'], 'config', 'generated_services', f'filtered_result_numbers_{month1}.json')
with open(file, 'w', encoding='utf-8') as f:
    json.dump(filtered_response_list, f, ensure_ascii=False, indent=4)

In [14]:
# запись предварительно отформатированных услуг в формате
# Услуга | УслугаКод | Цена

with open(os.path.join(config['BASE_DIR'], 'config', 'new_services.txt'), 'a', encoding='utf-8') as file:
    for i, res in enumerate(filtered_response_list):
        file.write(f'-----{i}-----\n')
        raws = res['СтрокиСчета']
        files = res['ФайлыСчета']
    
        for raw in raws:
            service = raw['Услуга']
            code = raw['УслугаКод']
            price = raw['Цена']
            s = f"{service}|{code}|{price}\n"
            file.write(s)
        file.write('file:\n')
        file.write(files[0])
        file.write('\n\n')

... ЗАМЕНА ЦЕНЫ НА КОММЕНТАРИЙ ...

### Загрузка отфильтрованного и дополненного "комментариями" файла

In [40]:
with open(os.path.join(config['BASE_DIR'], 'config', 'new_services.txt'), 'r', encoding='utf-8') as file:
    data = file.readlines()

In [41]:
# Удаление пустых строк из новых сервисов

data = list(filter(lambda x: bool(x), map(lambda y: y.strip(), data)))

In [42]:
reg_data = []
regex = r'.*\|.*\|.*'
for i in data:
    if re.fullmatch(regex, i):
        if not re.sub(r'\W', '', i).isdigit():
            reg_data.append(i)

In [43]:
len(reg_data)

205

In [44]:
def create_dataframe(data_list):
    # Преобразуем список в список списков, разделяя строки по символу |
    data_split = [item.split('|') for item in data_list]
    
    # Создаем DataFrame с соответствующими заголовками столбцов
    df = pd.DataFrame(data_split, columns=['service', 'code', 'comment'])
    
    return df

In [45]:
df = create_dataframe(reg_data)

In [46]:
df.tail()

,service,code,comment
200,Транспортно-Экспедиторское обслуживание,000000510,ранспортно-экспедиторское обслуживание По марш...
201,Прием и выдача грузов,Т1319,Прием и выдача грузов(прием и раскредит.по ст-...
202,Лабораторная энтомологическая экспертиза средн...,ТК-009110,Лабораторная энтомологическая экспертиза средн...
203,"Просмотр для выявления семян сорных растений, ...",ТК-008932,+
204,Лабораторная энтомологическая экспертиза средн...,ТК-009436,Лабораторная энтомологическая экспертиза средн...


In [47]:
# замена + (как в 1 в 1)

def replace_plus(raw):
    if raw['comment'] == '+':
        raw['comment'] = raw['service']
    return raw

In [48]:
df.apply(replace_plus, axis=1)
df.tail()

,service,code,comment
200,Транспортно-Экспедиторское обслуживание,000000510,ранспортно-экспедиторское обслуживание По марш...
201,Прием и выдача грузов,Т1319,Прием и выдача грузов(прием и раскредит.по ст-...
202,Лабораторная энтомологическая экспертиза средн...,ТК-009110,Лабораторная энтомологическая экспертиза средн...
203,"Просмотр для выявления семян сорных растений, ...",ТК-008932,"Просмотр для выявления семян сорных растений, ..."
204,Лабораторная энтомологическая экспертиза средн...,ТК-009436,Лабораторная энтомологическая экспертиза средн...


In [49]:
df = df.drop_duplicates().reset_index(drop=True)
df.tail()

,service,code,comment
171,Пломбировка грузового отсека автомобиля,ТК-009923,Пломбировка грузового отсека автомобиля
172,Транспортно-Экспедиторское обслуживание,000000510,ранспортно-экспедиторское обслуживание По марш...
173,Прием и выдача грузов,Т1319,Прием и выдача грузов(прием и раскредит.по ст-...
174,Лабораторная энтомологическая экспертиза средн...,ТК-009110,Лабораторная энтомологическая экспертиза средн...
175,Лабораторная энтомологическая экспертиза средн...,ТК-009436,Лабораторная энтомологическая экспертиза средн...


In [50]:
# порядок полей

df = df[['service', 'comment', 'code']]

In [52]:
df.tail()

,service,comment,code
171,Пломбировка грузового отсека автомобиля,Пломбировка грузового отсека автомобиля,ТК-009923
172,Транспортно-Экспедиторское обслуживание,ранспортно-экспедиторское обслуживание По марш...,000000510
173,Прием и выдача грузов,Прием и выдача грузов(прием и раскредит.по ст-...,Т1319
174,Лабораторная энтомологическая экспертиза средн...,Лабораторная энтомологическая экспертиза средн...,ТК-009110
175,Лабораторная энтомологическая экспертиза средн...,Лабораторная энтомологическая экспертиза средн...,ТК-009436


### Export to Excel

In [53]:
df.to_excel(os.path.join(config['BASE_DIR'], 'config','generated_services.xlsx'), index=False)

### Проверка на похожие строки

In [ ]:
for i in df['comment']:
    lst = list(dict.fromkeys(df['comment'].to_list()))
    lst.remove(i)
    dif = difflib.get_close_matches(i, lst, n=1)
    if dif:
        print(i)
        print(dif[0])
        print('----------------------')

### Дополнительно

In [ ]:
grouped = df.groupby(["service"]).agg(com=('comment', lambda x: list(x))).reset_index()

In [ ]:
grouped[grouped['com'].apply(len) > 1]

In [ ]:
df[df['service'] == 'Сверхнорм. использование контейнера']